In [ ]:
#from thllib import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
#from thllib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import networkx as nx


import flylib as flb
#from thllib import flylib as flb
import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
from flylib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import pandas as pd
import networkx as nx

from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [3]:
flynumbers = list(range(1389,1402))
fly_nums = range(1389,1392)
multi_fly_df = util.construct_multi_fly_df(fly_nums)



Making dataframe for fly number 1389
Making dataframe for fly number 1390
Making dataframe for fly number 1391


In [40]:
flynumbers = list(range(1389,1402))

fly_nums = range(1389,1392)#,1545)
#fly_nums = range(1635, 1637) + range(1637, 1639) + range(1640, 1643) + range(1645, 1647) + range(1648, 1651) + range(1653, 1654) #+ range(1655, 1656) 
flylist = [flb.NetFly(fnum,rootpath='/media/imager/FlyDataD/FlyDB/') for fnum in flynumbers]
l = [fly.open_signals() for fly in flylist]

In [17]:
### to extract motifs from closed loop data
#import signal

In [4]:
multi_fly_df.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg2_right,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right
0,0.000000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
1,0.020000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
2,0.040001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
3,0.060001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
4,0.080002,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0


In [11]:
#multi_fly_df.head()

In [5]:
def normd (muscle_data):
    x = muscle_data
    normalized = (x-min(x))/(max(x)-min(x))
    return normalized

In [37]:
fly = 1389

In [41]:
sorted_keys = sorted(fly.spikestates.keys())

cull_list = [('left', 'bkg'),('right', 'bkg'),
            ('left', 'iii24'),('right', 'iii24'),
            ('left', 'nm'),('right', 'nm'),
            ('left', 'pr'),('right', 'pr'),
            ('left', 'tpd'),('right', 'tpd')]

for cull in cull_list:
    sorted_keys.remove(cull)
#[sorted_keys.remove(cull) for cull in cull_list]

flydict = {}
for fly in flylist:
    flydict[fly.flynum] = fly
    
stim_conditions = [('common', 'idx', 'cl_blocks, g_x=-1, g_y=0 b_x=0, b_y=0'),
                  ('common', 'idx', 'cl_blocks, g_x=-1, g_y=0 b_x=-8, b_y=0'),
                  ('common', 'idx', 'cl_blocks, g_x=-1, g_y=0 b_x=8, b_y=0')]

In [6]:
print(multi_fly_df.columns.values)
print(np.unique(multi_fly_df['stimulus']))

idx = (multi_fly_df['stimulus']=='cl_blocks, g_x=-1, g_y=0, b_x=-8, b_y=0, ch=True')& \
        ((multi_fly_df['amp_diff']>0.1)&(multi_fly_df['amp_diff']<0.104))


['t' 'stimulus' 'amp_diff' 'left_amp' 'right_amp' 'fly_num' 'wbf'
 'experimental_condition' 'pr_left' 'tpd_left' 'nm_left' 'iii1_left'
 'iii3_left' 'i2_left' 'hg2_left' 'hg3_left' 'hg1_left' 'i1_left'
 'bkg_left' 'b1_left' 'b2_left' 'b3_left' 'iii24_left' 'hg4_left'
 'pr_right' 'tpd_right' 'nm_right' 'iii1_right' 'iii3_right' 'i2_right'
 'hg2_right' 'hg3_right' 'hg1_right' 'i1_right' 'bkg_right' 'b1_right'
 'b2_right' 'b3_right' 'iii24_right' 'hg4_right']
['cl_blocks, g_x=-1, g_y=-4 b_x=-8, b_y=0'
 'cl_blocks, g_x=-1, g_y=-4 b_x=0, b_y=0'
 'cl_blocks, g_x=-1, g_y=-4 b_x=8, b_y=0'
 'cl_blocks, g_x=-1, g_y=0 b_x=-8, b_y=0'
 'cl_blocks, g_x=-1, g_y=0 b_x=0, b_y=0'
 'cl_blocks, g_x=-1, g_y=0 b_x=8, b_y=0'
 'cl_blocks, g_x=-1, g_y=4 b_x=-8, b_y=0'
 'cl_blocks, g_x=-1, g_y=4 b_x=0, b_y=0'
 'cl_blocks, g_x=-1, g_y=4 b_x=8, b_y=0'
 'ol_blocks, g_x=-12, g_y=0 b_x=0, b_y=0'
 'ol_blocks, g_x=0, g_y=-4 b_x=0, b_y=0'
 'ol_blocks, g_x=0, g_y=0 b_x=0, b_y=0'
 'ol_blocks, g_x=0, g_y=4 b_x=0, b_y=0'
 '

In [8]:
multi_fly_df['l_plus_r'] = multi_fly_df['left_amp'] + multi_fly_df['right_amp']
all_stim_df = multi_fly_df


In [9]:
def rad_to_deg (rad):
    r_deg = np.rad2deg(((rad)/10) *2*np.pi)
    return r_deg

In [10]:
def key_to_key(inkey):
    """map the input key from the cov_mtrx_triang_layout.svg into a set of keys 
    (a tuple of tuples) that can be used to construct signals from the fly data"""
    try:
        k1 = {'R':'right','L':'left'}[inkey[0]]
    except KeyError:
        if inkey[0] == 'w':
            return ('common','wb_freq')
        else:
            return
    k2 = inkey[1:]
    return (k1,k2)

In [21]:
#from scipy import medfilt

In [11]:
from scipy.signal import medfilt

In [12]:
def median_filter_all_columns(dataframe):
    df = pd.DataFrame()
    dataframe = dataframe
    columns = dataframe.columns
    for column in columns:
        df[column]=scipy.signal.medfilt(dataframe[column], 3)
    return df

In [13]:
multi_fly_df = median_filter_all_columns(multi_fly_df)

In [18]:
mini_test = multi_fly_df[0:300]

In [23]:
mini_test.replace([np.inf, -np.inf], np.nan, inplace=True)
mini_test = mini_test.interpolate(method ='linear', limit_direction ='forward', limit=5)
mini_test = mini_test.fillna(0)

In [24]:
mini_test.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right,l_plus_r
0,0.000000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
1,0.020000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
2,0.040001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
3,0.060001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
4,0.080002,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0


In [ ]:
#multi_fly_df = multi_fly_df.replace([np.inf, -np.inf], np.nan, inplace=True)
#multi_fly_df_i = multi_fly_df.interpolate


In [ ]:
mutli_fly_df 

In [24]:
df = median_filter_all_columns(multi_fly_df)

In [25]:
df.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg2_right,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right
0,0.000000,pretrial_stripe_fix,NaN,NaN,NaN,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
1,0.020000,pretrial_stripe_fix,NaN,NaN,NaN,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
2,0.040001,pretrial_stripe_fix,NaN,NaN,NaN,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
3,0.060001,pretrial_stripe_fix,NaN,NaN,NaN,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
4,0.080002,pretrial_stripe_fix,NaN,NaN,NaN,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0


In [ ]:
## normalize med filtered 

In [ ]:
#normalized = normed_multi_fly_df.loc[normed_multi_fly_df['stimulus']== 'yaw_left, g_x=12, g_y=0, b_x=0, b_y=0, ch=0']


In [25]:
multi_fly_df.replace([np.inf, -np.inf], np.nan, inplace=True)
multi_fly_df_i = multi_fly_df.interpolate(method ='linear', limit_direction ='forward', limit=5)
multi_fly_df_i = multi_fly_df_i.fillna(0)

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right,l_plus_r
0,0.000000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
1,0.020000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
2,0.040001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
3,0.060001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
4,0.080002,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0


In [29]:
multi_fly_df_i.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right,l_plus_r
0,0.000000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
1,0.020000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
2,0.040001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
3,0.060001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0
4,0.080002,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0,0.0


In [30]:

def normd(fly_num, muscle):
        df=multi_fly_df_i.loc[multi_fly_df_i['fly_num']==fly_num]
        a = len(df[muscle])
        m = df[muscle]
        #a_5p = a*5/100.
        #a_5pr = round(a_5p)
        #minimums_5p = arr.argsort()[:a_5pr]
        #maximums_5p = arr.argsort()[:-a_5pr]
        #minimums_5p = np.argpartition(df, a_5pr)
        print("1.5th percentile:   ", np.percentile(m, 1.5))
        muscle_min = np.percentile(m, 1.5)
        print("98.5th percentile:   ", np.percentile(m, 98.5))
        muscle_max = np.percentile(m, 98.5)

        normalized = (m-muscle_min)/(muscle_max-muscle_min)
        return normalized

In [ ]:
normed_multi_fly_df = pd.DataFrame()

for flynum in fly_nums:
    df=multi_fly_df_i.loc[multi_fly_df_i['fly_num']==flynum]
    for key in sorted_keys: 
        a = normd(df[key])
        df[key] = a
    normed_multi_fly_df = normed_multi_fly_df.append(df)

In [44]:
muscle_list = ('b1_left', 'b2_left', 'b3_left', 'i1_left', 'i2_left', 'iii1_left', 'iii3_left', 'hg1_left', 'hg2_left', 'hg3_left', 'hg4_left', 
              'b1_right', 'b2_right', 'b3_right', 'i1_right', 'i2_right', 'iii1_right', 'iii3_right', 'hg1_right', 'hg2_right', 'hg3_right', 'hg4_right')



normed_multi_fly_df = pd.DataFrame()

for flynum in fly_nums:
    df=multi_fly_df_i.loc[multi_fly_df_i['fly_num']==flynum]
    for muscle in muscle_list: 
        a = normd(df[muscle])
        df[muscle] = a
    normed_multi_fly_df = normed_multi_fly_df.append(df)

In [47]:
normed_multi_fly_df = pd.DataFrame()

for flynum in fly_nums:
    df=multi_fly_df_i.loc[multi_fly_df_i['fly_num']==flynum]
    for muscle in muscle_list: 
        a = normd(flynum, muscle)
        df[muscle] = a
    normed_multi_fly_df = normed_multi_fly_df.append(df)

('1.5th percentile:   ', 0.15166135551371679)
('98.5th percentile:   ', 5.2186986985262234)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 0.43860766764520165)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 7.6465152012978974)


/home/imager/.local/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 4.5150823916209379)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 1.9143579215070765)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 0.96075196966886844)
('1.5th percentile:   ', 0.14458011608078972)
('98.5th percentile:   ', 7.0168113813132491)
('1.5th percentile:   ', 0.28308565017939552)
('98.5th percentile:   ', 12.824630017923651)
('1.5th percentile:   ', 1.3303575942437094)
('98.5th percentile:   ', 22.403472750516915)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 0.1508721704072003)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 0.12213600887507366)
('1.5th percentile:   ', 0.39396165468128419)
('98.5th percentile:   ', 5.8083277348564888)
('1.5th percentile:   ', 0.011416316489858244)
('98.5th percentile:   ', 0.955345993468942)
('1.5th percentile:   ', 1.1453639354382161)
('98.5th percentile:   ', 10.27546885755989)
('1.5th percentile:   ', 0.0)
('98.5th percentile:   ', 4.43186094240

In [49]:
normed_multi_fly_df.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right,l_plus_r
0,0.000000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,0.207107,0.124072,22.93889,0.005551,0.289151,0.001809,0.0,0.0,0.0
1,0.020000,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,0.207107,0.124072,22.93889,0.005551,0.289151,0.001809,0.0,0.0,0.0
2,0.040001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,0.207107,0.124072,22.93889,0.005551,0.289151,0.001809,0.0,0.0,0.0
3,0.060001,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,0.207107,0.124072,22.93889,0.005551,0.289151,0.001809,0.0,0.0,0.0
4,0.080002,pretrial_stripe_fix,0.0,0.0,0.0,1389.0,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,0.0,0.207107,0.124072,22.93889,0.005551,0.289151,0.001809,0.0,0.0,0.0


In [50]:
normed_multi_fly_df.to_csv("normed_multi_fly_df_CL_data_for_UMAP.csv")